# wps_generate_prsn

#### wps_generate_prsn is a process that runs the [generate_prsn](https://github.com/pacificclimate/climate-explorer-data-prep#generate_prsn-generate-snowfall-file) module of PCIC Climate Explorer Data Preparation Tools. Here, the client will try to connect to a remote Thunderbird instance using the url parameter.

In [1]:
from birdy import WPSClient
import requests
import os
from wps_tools.utils import copy_http_content
from netCDF4 import Dataset
from tempfile import NamedTemporaryFile


docker_url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thunderbird/wps'
url = os.environ.get('WPS_URL', docker_url)
thunderbird = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [2]:
# NBVAL_IGNORE_OUTPUT
thunderbird.generate_prsn?

Signature:
thunderbird.generate_prsn(
    prec,
    tasmin=None,
    tasmax=None,
    dry_run=None,
    chunk_size=100,
    output_file='None',
    loglevel='INFO',
)
Docstring:
Generate precipitation as snow file from precipitation and minimum/maximum temperature data

Parameters
----------
prec : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Precipitation file to process
tasmin : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmin file to process
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Tasmax file to process
chunk_size : integer
    Number of time slices to be read/written at a time
output_file : string
    Optional custom name of output file
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
dry_run : boolean
    Checks file to ensure compatible with process

Returns
-------
output : ComplexData:mimetype:`applic

#### We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# Dry-run
pr_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/pr_day_BCCAQv2+ANUSPLIN300_NorESM1-M_historical+rcp26_r1i1p1_19500101-19500107.nc"
tasmin_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/tasmin_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"
tasmax_file_opendap = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/storage/data/projects/comp_support/daccs/test-data/tasmax_day_BCCAQv2%2BANUSPLIN300_NorESM1-M_historical%2Brcp26_r1i1p1_19500101-19500107.nc"

dry_output = thunderbird.generate_prsn(pr_file_opendap, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=True)

In [4]:
req = requests.get(dry_output.get()[0])
dependent_varnames = [line.split(":")[-1].strip(" ['']") for line in req.content.decode('utf-8').split("\n") if "dependent_varnames" in line]
assert dependent_varnames == ['pr', 'tasmin', 'tasmax']

In [5]:
# Normal run
output = thunderbird.generate_prsn(pr_file_opendap, tasmin_file_opendap, tasmax_file_opendap, chunk_size=50, dry_run=False, output_file="prsn_test_mixed.nc")

In [6]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output.get()[0], tmp_file))
    assert 'prsn' in output_data.variables.keys()